In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from wdtools import *

In [3]:
sets = list(map(lambda x: 'Set00'+x, ['1', '2', '3', '4']))

In [4]:
nm2add = [0, 1420, 2143, 2878]

In [5]:
dat = []

In [6]:
for setid in sets:
    df = combine_wd_tables(setID=setid, nm_to_add=nm2add[sets.index(setid)])
    df.loc[:, 'SetID'] = setid
    dat.append(df)

In [7]:
data=pd.concat(dat)

In [8]:
data.shape

(3932, 26)

In [9]:
data.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'SetID'],
      dtype='object')

In [10]:
data.loc[data.county=='TIllamook', 'county'] = 'Tillamook'

In [11]:
ndf = data.groupby('county', as_index=False)['wetdet_delin_number'].nunique()

In [12]:
ndf.rename(columns={'wetdet_delin_number': 'total'}, inplace=True)

In [13]:
qaqc = []
unmt = []
issue = []
mapped = []

In [14]:
qaqcdf = pd.DataFrame(columns=['SetID', 'Counties', 'Total Count', 'Total QAQC Count', 
                               'Partial and Mythical Count', 'Unmatched Count', 'Issue ID Count', 'Mapped Count'])

In [15]:
for setid in sets:
    print(setid)
    wd_dt = data[data.SetID==setid]
    counties = wd_dt['county'].unique()
    print(counties)
    N = data[data.SetID==setid]['wetdet_delin_number'].nunique()
    print(N)
    matched = gpd.read_file(os.path.join(inpath + '\\output\\matched\\', f'matched_records_{setid}_edited.shp'))
    final = gpd.read_file(os.path.join(inpath + '\\output\\final\\', f'mapped_wd_{setid}.shp'))
    a = list(matched[matched.notes.notnull()].wdID.unique())
    unmatched = pd.read_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setid}_2.csv'))
    b = list(unmatched.wetdet_delin_number.unique())
    unmatched_wd = [x for x in b if x not in a]
    unmatched3 = unmatched[unmatched.wetdet_delin_number.isin(unmatched_wd)]
    unmatched3.to_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setid}_3.csv'), index=False)
    unmt.append(unmatched_wd)
    qaqc.append(unique(a+b))
    issues = pd.read_csv(os.path.join(inpath, "output", "to_review", f"{setid}_Mapping_Issues.csv"))
    issueIDs = list(issues.wetdet_delin_number.unique())
    issue.append(issueIDs)
    c = list(final[final.code==1].wdID.unique())
    mapped.append(c)
    to_check = [wid for wid in unique(a+b) if wid not in (issueIDs+c)]
    print(f"""partial and mythical count - {len(a)}, 
          unmatched count - {len(unmatched_wd)},
          total QAQC count - {len(unique(a+b))},
          issue ID count -{len(issueIDs)},
          mapped count - {len(c)},
          total count - {N},
          to check - {to_check}""")
    qaqcdf = qaqcdf.append({'SetID': setid,
                            'Counties': ', '.join(list(counties)),
                            'Total Count': N,
                            'Total QAQC Count': len(unique(a+b)), 
                            'Partial and Mythical Count': len(a), 
                            'Unmatched Count': len(unmatched_wd),
                            'Issue ID Count': len(issueIDs),
                            'Mapped Count': len(c)}, ignore_index=True)

Set001
['Clatsop' 'Coos' 'Deschutes' 'Harney' 'Jackson' 'Josephine' 'Lake'
 'Marion']
1003
partial and mythical count - 264, 
          unmatched count - 14,
          total QAQC count - 278,
          issue ID count -4,
          mapped count - 277,
          total count - 1003,
          to check - []
Set002
['Baker' 'Benton' 'Clackamas' 'Morrow']
434
partial and mythical count - 138, 
          unmatched count - 14,
          total QAQC count - 152,
          issue ID count -15,
          mapped count - 141,
          total count - 434,
          to check - []
Set003
['Columbia' 'Grant' 'Klamath' 'Lincoln' 'Tillamook']
433
partial and mythical count - 127, 
          unmatched count - 11,
          total QAQC count - 138,
          issue ID count -9,
          mapped count - 123,
          total count - 433,
          to check - ['WD2019-0407', 'WD2019-0482', 'WD2019-0586', 'WD2021-0074', 'WD2021-0498', 'WD2022-0124']
Set004
['Crook' 'Curry' 'Hood River' 'Jefferson' 'Linn' 'Multnoma

In [16]:
#review_loop_r1(wdid_list=['WD2019-0407', 'WD2019-0482', 'WD2019-0586', 'WD2020-0109', 'WD2021-0074', 
#                          'WD2021-0290', 'WD2021-0498', 'WD2022-0124', 'WD2022-0200', 'WD2022-0231'], partial=True, idx=True, wddf=data)

In [17]:
file=os.path.join(inpath, 'reporting', f'SetStatusReport{str(date.today()).replace("-", "")}.xlsx')

In [18]:
qaqcdf.to_excel(file, index=False)

In [19]:
reformat(file)

Removed format
Autofitted columns...


In [20]:
qaqc = unique(np.concatenate(qaqc))

In [21]:
len(qaqc)

789

In [22]:
df = data[['wetdet_delin_number', 'county']]

In [23]:
wdcnt_dict = df.set_index('wetdet_delin_number').to_dict(orient='dict')['county']

In [24]:
# count QAQC by county
qaqc_df = count_lst_ele(wdcnt_dict, qaqc, 'QAQC')

In [25]:
colnm='QAQC_comp'

In [26]:
comp_df = count_lst_ele(wdcnt_dict, flatten(mapped)+flatten(issue), colnm)

In [27]:
out = ndf.merge(qaqc_df, on='county', how='left')

In [28]:
out1 = out.merge(comp_df, on='county', how='left')

In [29]:
sel1 = out1.QAQC_comp.isnull()
out1.loc[sel1, colnm] = 0

In [30]:
set4cty = ['Crook', 'Curry', 'Hood River', 'Jefferson', 'Linn', 'Multnomah', 'Wasco', 
           'Yamhill', 'Yamhill and Washington']

In [31]:
sel2 = (out1.QAQC_comp > out1.QAQC) | (~out1.county.isin(set4cty))

In [32]:
out1.loc[sel2, colnm] = out1.loc[sel2, 'QAQC']

In [33]:
out1[colnm] = out1[colnm].apply(np.int64)

In [34]:
out1['QAQC_pct'] = out1.apply(lambda row: str(round(row.QAQC/row.total*100,1))+('%'), axis=1)

In [35]:
out1['QAQC_comp_pct'] = out1.apply(lambda row: str(round(row.QAQC_comp/row.QAQC*100,1))+('%'), axis=1)

In [36]:
out1

,county,total,QAQC,QAQC_comp,QAQC_pct,QAQC_comp_pct
0,Baker,38,12,12,31.6%,100.0%
1,Benton,91,30,30,33.0%,100.0%
2,Clackamas,275,94,94,34.2%,100.0%
3,Clatsop,137,41,41,29.9%,100.0%
4,Columbia,93,33,33,35.5%,100.0%
5,Coos,120,27,27,22.5%,100.0%
6,Crook,35,11,11,31.4%,100.0%
7,Curry,17,3,3,17.6%,100.0%
8,Deschutes,172,38,38,22.1%,100.0%
9,Grant,23,6,6,26.1%,100.0%


In [37]:
out1.columns = ['County', 'Total Count', 'QAQC Count', 'QAQC Completed Count', 
                'QAQC Count Percentage', 'QAQC Completed Count Percentage']

In [38]:
file=os.path.join(inpath, 'reporting', f'StatusReport{str(date.today()).replace("-", "")}.xlsx')

In [39]:
out1.to_excel(file, index=False)

In [40]:
reformat(file)

Removed format
Autofitted columns...
